In [ ]:
X = X.merge(hotels, how="left").merge(operators, how="left")

In [1]:
from datetime import date, timedelta

import pandas as pd
import requests
from config import Config
from db import *
from requests_ntlm import HttpNtlmAuth

In [2]:
date_fr = date.today()
date_to = date_fr + timedelta(days=13)

In [3]:
ssrs_url = "http://95.211.42.206/ReportServer_SSRS?/OM Reports/Reservations/PowerQ-reservations"
ssrs_usr = Config.SSRS_USERNAME
ssrs_pwd = Config.SSRS_PASSWORD
payload = [
    ("maxprocessdate_from:isnull", True),
    ("maxprocessdate_to:isnull", True),
    ("arrival_date_from", date_fr.strftime("%m/%d/%Y %H:%M:%S")),
    ("arrival_date_to", date_to.strftime("%m/%d/%Y %H:%M:%S")),
    ("Inhousedatefrom:isnull", True),
    ("InhousedateTo:isnull", True),
    ("operatorgroup", 5),
    ("rs:ParameterLanguage", ""),
    ("rs:Command", "Render"),
    ("rs:Format", "CSV"),
    ("rc:ItemPath", "table1"),
]

params = urllib.parse.urlencode(payload, quote_via=urllib.parse.quote)

In [4]:
response = requests.get(
    ssrs_url,
    params=params,
    stream=True,
    auth=HttpNtlmAuth(ssrs_usr, ssrs_pwd),
)

In [5]:
data = response.content.decode("utf8")

In [6]:
df = pd.read_csv(
    io.StringIO(data),
    usecols=[1, 2, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19],
    header=0,
    names=[
        "status",
        "operator",
        "ref_id",
        "hotel_id",
        "res_id",
        "bkg_ref",
        "hotel_name",
        "guest_name",
        "in_date",
        "out_date",
        "room_description",
        "meal",
        "days",
        "adult",
        "child",
        "Infant",
    ],
)

In [7]:
orascom_hotels = [202191, 195820, 193348]

In [8]:
df = df[df["hotel_id"].isin(orascom_hotels)]
df["status"] = df["status"].str.capitalize()
df = df[df["status"] != "Can"]

In [9]:
outfile = f'Orascom_{date_fr.strftime("%Y_%m_%d")}_{date_to.strftime("%Y_%m_%d")}.xlsx'
df.drop(["ref_id", "res_id", "hotel_id"], axis=1, inplace=True)
df.to_excel(outfile, index=False)

In [10]:
import io
from datetime import datetime, timedelta

import pandas as p
import pandas as pd
import requests
from config import Config
from db import *
from requests_ntlm import HttpNtlmAuth
from sklearn.base import BaseEstimator, TransformerMixin


class BookingData:
    def __init__(self, ssrs_url, ssrs_usr, ssrs_pwd):
        self.ssrs_url = ssrs_url
        self.ssrs_usr = ssrs_usr
        self.ssrs_pwd = ssrs_pwd

    def set_params(self, date_fr, date_to):
        self.params = {
            "from": date_fr.strftime("%m/%d/%Y %H:%M:%S"),
            "to": date_to.strftime("%m/%d/%Y %H:%M:%S"),
            "d1:isnull": True,
            "d2:isnull": True,
            "MaxProcessDate_from:isnull": True,
            "MaxProcessDate_to:isnull": True,
            "ReportParameter1": True,
            "RefIDs:isnull": True,
            "rs:ParameterLanguage": "",
            "rs:Command": "Render",
            "rs:Format": "CSV",
            "rc:ItemPath": "table1",
        }

    def get(self):
        response = requests.get(
            self.ssrs_url,
            params=self.params,
            stream=True,
            auth=HttpNtlmAuth(self.ssrs_usr, self.ssrs_pwd),
        )

        if response.status_code == 200:
            data = response.content.decode("utf8")
            if len(data) > 424:
                return data
        return None


class BookingReport(BaseEstimator, TransformerMixin):
    def __init__(
        self,
    ):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        hotels = pd.read_sql(
            sql=select(
                hotel_mapping.c.provider_id.label("hotel_id"),
                hotel_mapping.c.hotel_name.label("hotel_name"),
            ),
            con=engine.connect(),
        )

        operators = pd.read_sql(
            sql=select(
                operator.c.id.label("operator_id"),
                operator.c.short_name.label("operator_name"),
            ),
            con=engine.connect(),
        )

        orascom_hotels = [202191, 195820, 193348]
        X = X[X["hotel_id"].isin(orascom_hotels)]
        X.loc[:, "status"] = X["status"].str.capitalize()
        X = X[X["status"] != "Can"]

        X = X.merge(hotels, how="left").merge(operators, how="left")

        X.drop(
            [
                "hotel_id",
                "operator_id",
                "operator_code",
                "purchase",
                "sales",
                "purchase_id",
                "purchase_code",
                "sales_id",
                "sales_code",
                "updated_at",
                "ref_id",
                "res_id",
                "created_at",
                "status",
            ],
            axis=1,
            inplace=True,
        )

        X = X[
            [
                "operator_name",
                "bkg_ref",
                "hotel_name",
                "guest_name",
                "sales_date",
                "in_date",
                "out_date",
                "room",
                "meal",
                "adult",
                "child",
                "days",
            ]
        ]

        X.sort_values(["hotel_name", "in_date"], inplace=True)

        return X


date_fr = datetime.now().replace(hour=0, minute=0, second=0)
date_to = date_fr + timedelta(days=13)

ssrs_url = (
    "http://95.211.42.206/ReportServer_SSRS?/OM Reports/Reservations/Bookings Data"
)
ssrs_usr = Config.SSRS_OM_USERNAME
ssrs_pwd = Config.SSRS_OM_PASSWORD

booking_data = BookingData(ssrs_url, ssrs_usr, ssrs_pwd)
booking_data.set_params(date_fr, date_to)
data = booking_data.get()

from preprocessor import CsvReader, DataEncoder
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ("csv_reader", CsvReader()),
        ("encoder", DataEncoder()),
        ("booking_report", BookingReport()),
    ]
)

df = pipeline.fit_transform(data)

from email_sender import EmailSender
from excel_writer import ExcelWriter

outfile = f'Orascom_{date_fr.strftime("%Y_%m_%d")}_{date_to.strftime("%Y_%m_%d")}.xlsx'
excel_writer = ExcelWriter([df], outfile)
excel_writer.writer.close()

sender = EmailSender(
    subject="Orascom Weekly Arrival",
    to=[
        "ivan.orara@meetingpointuae.com",
    ],
)

sender.send_email([outfile])

In [ ]:
stmt = select(reservation).where(
    func.date(reservation.c.in_date) >= date_fr,
    func.date(reservation.c.in_date) < date_to,
)